# ControlNet Direct

In [1]:
%load_ext autoreload
%autoreload 2

from lib_prior.diffusion.ctrl_sd_prior import CtrSDPrior

prior = CtrSDPrior()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [ ]:
# import imageio
# from matplotlib import pyplot as plt
# import torch, numpy as np

# img_fn = "./data/DAVIS/train/images/00000.jpg"
# mask_fn = "./data/DAVIS/train/epipolar_mask/00000.jpg"

# img_np = imageio.imread(img_fn)
# img_np = img_np.astype(np.float32) / 255.0
# mask_np = 1.0 - imageio.imread(mask_fn).astype(np.float32) / 255.0
# L = (img_np.shape[1] - img_np.shape[0]) //2
# img_np = img_np[:, L:L+img_np.shape[0], :]
# mask_np = mask_np[:, L:L+img_np.shape[0]]
# # plt.imshow(mask_np)
# print(mask_np.shape)
# print(mask_np.max())
# print(img_np.shape)

# mask_np = np.random.rand(*mask_np.shape).reshape(*mask_np.shape) > 0.4

# inpainted = prior.inpaint(
#     img_np,
#     # np.zeros_like(mask_np),
#     mask_np,
#     "realistic and clear crop of a photo",
#     diff_steps=50,
# )

# fig = plt.figure(figsize=(12, 4))
# plt.subplot(1, 3, 1)
# plt.imshow(img_np), plt.title("Input"), plt.axis("off")
# plt.subplot(1, 3, 2)
# plt.imshow(mask_np), plt.title("Mask"), plt.axis("off")
# plt.subplot(1, 3, 3)
# plt.imshow(inpainted), plt.title("Output"), plt.axis("off")

In [2]:
import torch
import os, os.path as osp
import logging
from lib_4d.solver_gs import Solver
from lib_4d.gs_static_model import StaticGaussian
from lib_4d.gs_dyn_model import DenseDynGaussian
from lib_4d.camera import SimpleFovCameras
from lib_4d.cfg_helpers import OptimCFG, GSControlCFG
from lib_prior.diffusion.sd_sds import StableDiffusionSDS

# load and render a novel view
src = "./data/DAVIS/train"
static_saved_dir = "data/DAVIS/train/log/20240117_143638"
dyn_saved_dir = "data/DAVIS/train/log/20240117_144846"
device = torch.device("cuda:0")

solver = Solver(
    src,
    device,
    # start_frame=35,
    # end_frame=55,
    depth_mode="zoe",  # "marigold"
    align_to_zoe=True,
    load_resize=1.0,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


| endurance | INFO | Jan-18-19:32:58 | Loading motion masks...   [solver_utils.py:117]
100%|██████████| 80/80 [00:00<00:00, 2353.16it/s]
| endurance | INFO | Jan-18-19:32:58 | Loading motion masks...   [solver_utils.py:117]
100%|██████████| 80/80 [00:00<00:00, 3486.25it/s]
| endurance | INFO | Jan-18-19:32:58 | Loading flows...   [solver_utils.py:58]
100%|██████████| 79/79 [00:00<00:00, 294.86it/s]
| endurance | INFO | Jan-18-19:32:58 | Loading rgbs...   [solver_utils.py:42]
100%|██████████| 80/80 [00:00<00:00, 268.77it/s]
| endurance | INFO | Jan-18-19:32:59 | Loading depths from ./data/DAVIS/train/zoe_depth ...   [solver_utils.py:102]
100%|██████████| 80/80 [00:00<00:00, 130.70it/s]
| endurance | INFO | Jan-18-19:32:59 | rgbs: (80, 480, 854, 3), depths: (80, 480, 854), flows: [158,(480, 854, 2)], motion_masks: (80, 480, 854)   [solver_utils.py:188]
| endurance | INFO | Jan-18-19:32:59 | Filtering depth maps...   [solver_utils.py:229]
100%|██████████| 80/80 [00:00<00:00, 384.88it/s]
|

In [3]:
model_fn = osp.join(static_saved_dir, "joint_model.pth")
cam_fn = osp.join(static_saved_dir, "joint_cams.pth")
st_model = StaticGaussian(load_fn=model_fn).to(solver.device)
cams: SimpleFovCameras = SimpleFovCameras(solver.T, 40.0)
cams.load_state_dict(torch.load(cam_fn), strict=True)
cams.to(solver.device)
dyn_model_fn = osp.join(dyn_saved_dir, "joint_dyn_model_1.0res.pth")
dyn_model = DenseDynGaussian(load_fn=dyn_model_fn).to(solver.device)

| endurance | INFO | Jan-18-19:33:01 | Loading static model from data/DAVIS/train/log/20240117_143638/joint_model.pth   [gs_static_model.py:68]
| endurance | INFO | Jan-18-19:33:01 | StaticGaussian: 218.3K points   [gs_static_model.py:109]
| endurance | INFO | Jan-18-19:33:01 | _xyz, 0.655M   [gs_static_model.py:112]
| endurance | INFO | Jan-18-19:33:01 | _features_dc, 0.655M   [gs_static_model.py:112]
| endurance | INFO | Jan-18-19:33:01 | _features_rest, 0.000M   [gs_static_model.py:112]
| endurance | INFO | Jan-18-19:33:01 | _opacity, 0.218M   [gs_static_model.py:112]
| endurance | INFO | Jan-18-19:33:01 | _scaling, 0.655M   [gs_static_model.py:112]
| endurance | INFO | Jan-18-19:33:01 | _rotation, 0.873M   [gs_static_model.py:112]
| endurance | INFO | Jan-18-19:33:01 | ------------------------------   [gs_static_model.py:113]
| endurance | INFO | Jan-18-19:33:01 | Loading dynamic model from data/DAVIS/train/log/20240117_144846/joint_dyn_model_1.0res.pth   [gs_dyn_model.py:75]
| end

In [ ]:
# from lib_4d.gs_viz_helpers import viz_scene, cat_gs
# from pytorch3d.transforms import quaternion_to_matrix
# from lib_4d.solver_utils import fovdeg2focal


# res=  512
# # gs5_param = dyn_model(40)
# # gs5_param = cat_gs(*gs5_param, *st_model())
# gs5_param = st_model()

# start_tid = 0
# end_tid = cams.T

# viz_R = quaternion_to_matrix(cams.q_wc[start_tid : end_tid + 1])
# viz_t = cams.t_wc[start_tid : end_tid + 1]
# viz_f = fovdeg2focal(90.0)
# _, viz_dict = viz_scene(
#     res, res, viz_R, viz_t, viz_f=viz_f, gs5_param=gs5_param, draw_camera_frames=True, return_full=True
# )
# print(viz_dict.keys())
# render_dict = viz_dict["scene_camera_20deg"]

In [20]:
import cv2
import numpy as np
import torch
from matplotlib import pyplot as plt

with torch.no_grad():
    # sample a camera
    left_R_wc, left_t_wc = cams.get_ith_Rt_wc(0)
    left_R_wc = left_R_wc.detach().clone()
    left_t_wc = left_t_wc.detach().clone()
    left_t_wc[0] = left_t_wc[0] + 1.5
    left_t_wc[2] = left_t_wc[2] -0.2
    
    # TODO: smart sample
    
    render_dict = solver.render_frame(
        cams.rel_focal,
        cams,
        ind=0,
        static_model=st_model,
        dynamic_model=dyn_model,
        render_view_id=0,
        square=True,
        R_cw=left_R_wc.T,
        t_cw=-left_R_wc.T @ left_t_wc,
        scale_factor=0.00001,
        opa_factor=1.0
    )
    
    render_mask_np = (render_dict["alpha"][0] < 0.5).cpu().numpy()
    
    render_dict = solver.render_frame(
        cams.rel_focal,
        cams,
        ind=0,
        static_model=st_model,
        dynamic_model=dyn_model,
        render_view_id=0,
        square=True,
        R_cw=left_R_wc.T,
        t_cw=-left_R_wc.T @ left_t_wc,
        # scale_factor=0.00001,
        # opa_factor=1.0
    )

rgb = render_dict["rgb"]
alpha = render_dict["alpha"]
# render_mask_np = (alpha[0] < 0.5).cpu().numpy()

dksize = 2
dilate_kernel = np.ones((dksize, dksize), np.uint8)
# render_mask_np = cv2.dilate(
#     render_mask_np.astype(np.uint8), dilate_kernel, iterations=1
# )
render_mask_np = cv2.erode(
    render_mask_np.astype(np.uint8), dilate_kernel, iterations=1
)

random_mask_np = (
    np.random.rand(*render_mask_np.shape).reshape(*render_mask_np.shape) > 2.0
)
# random_mask_np = cv2.dilate(
#     random_mask_np.astype(np.uint8), dilate_kernel, iterations=1
# )
mask_np = np.logical_or(render_mask_np, random_mask_np)


img_np = rgb.permute(1, 2, 0).cpu().numpy()
img_np = np.clip(img_np, 0.0, 1.0)

print(mask_np.mean())

inpainted = prior.inpaint(
    img_np,
    # np.ones_like(mask_np),
    # np.zeros_like(mask_np),
    mask_np,
    "smooth and realistic",
    # "realistic, sharp and clear crop of a photo",
    # "railway, village",
    diff_steps=100,
)

fig = plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(img_np), plt.title("Input"), plt.axis("off")
plt.subplot(1, 3, 2)
plt.imshow(mask_np), plt.title("Mask"), plt.axis("off")
plt.subplot(1, 3, 3)
plt.imshow(inpainted), plt.title("Output"), plt.axis("off")
plt.tight_layout()
plt.savefig("test.png")
plt.close()

0.7251649305555555


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
raise RuntimeError("stop here")

# SD SDS

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from lib_prior.diffusion.sd_sds import StableDiffusionSDS

prior = StableDiffusionSDS(
    torch.device("cuda:0"), fp16=True, vram_O=True, t_range=[0.8,0.99]
)

In [ ]:
import imageio
from matplotlib import pyplot as plt
import torch, numpy as np

img_fn = "./data/DAVIS/train/images/00000.jpg"
mask_fn = "./data/DAVIS/train/epipolar_mask/00000.jpg"

img_np = imageio.imread(img_fn)
img_np = img_np.astype(np.float32) / 255.0
mask_np = 1.0 - imageio.imread(mask_fn).astype(np.float32) / 255.0
L = (img_np.shape[1] - img_np.shape[0]) //2
img_np = img_np[:, L:L+img_np.shape[0], :]
mask_np = mask_np[:, L:L+img_np.shape[0]]
# plt.imshow(mask_np)
print(mask_np.shape)
print(mask_np.max())
print(img_np.shape)

In [ ]:
prior.get_text_embeds(
    ["a train is turning on the two side rails in a farm field"],
    [
        "ugly, bad anatomy, blurry, pixelated obscure, unnatural colors, poor lighting, dull, and unclear, lowres, low quality, artifacts, duplicate, morbid, mutilated, poorly drawn face, dehydrated, bad proportions"
    ],
)

In [ ]:
pred_rgb = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).cuda()
prior.train_step(pred_rgb)

In [ ]:
# load the trained model
